In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import numpy as np
import os
import math
from numpy import random
from PIL import Image
import torchvision
from torchvision import transforms, datasets
from collections import OrderedDict
import glob

In [42]:
# img_path = glob.glob('/home/b20saito/CODE/BENKYO/chainer/chapter03/train/*')
#print(len(img_path))
images = []

# 全てのファイル
fs = glob.glob('/home/b20saito/CODE/BENKYO/chainer/chapter03/train/*')
#fs = glob.glob('/home/b20saito/DATASETS/anime-face-dataset/data/*')
for fn in fs:
	# 画像を読み込み
	img = Image.open(fn).resize((320, 320)).convert('YCbCr')
	cur_x = 0
	while cur_x <= 320 - 40:
		cur_y = 0
		while cur_y <= 320 - 40:
			# 画像から切りだし
			rect = (cur_x, cur_y, cur_x+40, cur_y+40)
			cropimg = img.crop(rect).copy()
			# 配列に追加
			images.append(cropimg)
			# 次の切りだし場所へ
			cur_y += 20
		cur_x += 20


In [43]:
batch_size = 128

In [44]:
uses_device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(uses_device)

cuda:0


In [45]:
class SuperResolutionNN(nn.Module):
    def __init__(self):
        super(SuperResolutionNN, self).__init__()
        self.layer = nn.Sequential(OrderedDict([
            ("conv1",nn.Conv2d(1,56,kernel_size=5,stride=1,padding=0)),
            ("PR1",nn.PReLU()),
            ("conv2",nn.Conv2d(56,12,1,1,0)),
            ("PR2",nn.PReLU()),
            ("conv3",nn.Conv2d(12,12,3,1,1)),
            ("PR3",nn.PReLU()),
            ("conv4",nn.Conv2d(12,12,3,1,1)),
            ("PR4",nn.PReLU()),
            ("conv5",nn.Conv2d(12,12,3,1,1)),
            ("PR5",nn.PReLU()),
            ("conv6",nn.Conv2d(12,12,3,1,1)),
            ("PR6",nn.PReLU()),
            ("conv7",nn.Conv2d(12,56,1,1,1)),
            ("PR7",nn.PReLU()),
            ("Deconv",nn.ConvTranspose2d(56,1,9,3,4))
        ]))
        nn.init.normal_(self.layer.conv1.weight, 0.0, 0.0378)
        nn.init.normal_(self.layer.conv2.weight, 0.0, 0.3536)
        nn.init.normal_(self.layer.conv3.weight, 0.0, 0.1179)
        nn.init.normal_(self.layer.conv4.weight, 0.0, 0.189)
        nn.init.normal_(self.layer.conv4.weight, 0.0, 0.0001)
    
    def forward(self,x,t=None, train=True):
        # :param x: 入力値
        # :param t: 正解のラベル
        # :param train: 学習かどうか
        out = self.layer(x)
        #損失か結果を返す
        #差の絶対値の平均(平均二乗誤差)
        return out

In [46]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self,path,transform=None):
        self.transform = transform
        self.path = path
    def __len__(self):
        return len(self.path)

    def __getitem__(self,idx):
        img = Image.open(self.path[idx]).convert('YCbCr')
        
        #高解像度データ
        hpix = np.array(img).astype(np.float32) /255.0
        hp = hpix[:,:,0]
        #print('==HR==\n',hp.shape)
        #低解像度データ
        low = img.resize((16,16),Image.NEAREST)
        lpix = np.array(low).astype(np.float32) / 255.0
        lp = lpix[:,:,0]
        #print('==LR==\n',lp.shape)
        return np.expand_dims(hp,0),np.expand_dims(lp,0),

In [47]:
#データセットの作成と、呼び出し
transform = transforms.Compose([transforms.ToTensor])
dataset = MyDataset(img_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) #batch_size=128

In [48]:
#ニューラルネットワークを作成
model = SuperResolutionNN().to(uses_device)

In [49]:
#誤差逆電波法アルゴリズムを選択する
optimizer = optim.Adam(model.parameters())

In [ ]:
for data in dataloader:
    model(data[0])
    break

In [51]:
print(data[0].shape)
print(data[1].shape)
model.to("cpu")(data[0]).shape

torch.Size([128, 1, 320, 320])
torch.Size([128, 1, 16, 16])


torch.Size([128, 1, 952, 952])

In [ ]:
lossfc = nn.MSELoss()

In [ ]:
for i in range(10000):
    for lp,hp in dataloader:
        output = model(input)
        
        loss = lossfc(output,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
img = img_path[0]
print(img)
img = Image.open(img)
img = np.asarray(img)
plt.imshow(img)
plt.show()